In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Update the paths to point to your image directories with forward slashes
base_dir = r'C:/Users/matia/OneDrive/IFTS/MODELIZADO DE SISTEMAS DE IA/Modelizado/image_set'  # Base directory
train_dir = base_dir + r'/train'
val_dir = base_dir + r'/val'
test_dir = base_dir + r'/test'

In [24]:
# Image data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')


In [25]:
# Validation and test data generators without augmentation
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [26]:
# Load the training data with target size set to 256x256 (since your images are already that size)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(256, 256), 
                                                    batch_size=32, class_mode='categorical')


Found 2490 images belonging to 20 classes.


In [27]:
# Load the validation data with target size 256x256
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(256, 256), 
                                                batch_size=32, class_mode='categorical')


Found 533 images belonging to 20 classes.


In [28]:
# Load the Xception model as a base (pretrained on ImageNet)
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models


In [29]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # Freeze the pretrained model layers


In [30]:
# Build the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])


In [31]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
# Display the model summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 8, 8, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 20)             │        20,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,980,156 (87.66 MB)

 Trainable params: 2,118,676 (8.08 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Le subimos las epocas a 50 con un early stop de 3, (se puede subir a mas para tener mejor precision por lo que parece)

In [33]:
# Train the model
#history = model.fit(train_generator, epochs=5, validation_data=val_generator)

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator, epochs=50, validation_data=val_generator, 
                    callbacks=[early_stopping])

Epoch 1/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 609s 8s/step - accuracy: 0.4856 - loss: 1.7698 - val_accuracy: 0.7767 - val_loss: 0.6685
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 578s 7s/step - accuracy: 0.7440 - loss: 0.8563 - val_accuracy: 0.8143 - val_loss: 0.6095
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 572s 7s/step - accuracy: 0.7787 - loss: 0.7167 - val_accuracy: 0.8086 - val_loss: 0.5972
Epoch 4/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 566s 7s/step - accuracy: 0.7828 - loss: 0.6619 - val_accuracy: 0.8443 - val_loss: 0.5147
Epoch 5/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 591s 8s/step - accuracy: 0.8041 - loss: 0.6425 - val_accuracy: 0.8218 - val_loss: 0.5428
Epoch 6/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 639s 8s/step - accuracy: 0.8115 - loss: 0.5745 - val_accuracy: 0.8424 - val_loss: 0.5425
Epoch 7/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 617s 8s/step - accuracy: 0.8232 - loss: 0.5486 - val_accuracy: 0.8555 - val_loss: 0.5440


In [34]:
# Load and evaluate on test data with target size 256x256
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(256, 256), 
                                                  batch_size=32, class_mode='categorical')


Found 545 images belonging to 20 classes.


In [35]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


18/18 ━━━━━━━━━━━━━━━━━━━━ 103s 6s/step - accuracy: 0.8200 - loss: 0.5592
Test accuracy: 0.8385320901870728


In [36]:
# Predict on a single image
from tensorflow.keras.preprocessing import image
import numpy as np


In [42]:
img_path = r'C:/Users/matia/OneDrive/IFTS/MODELIZADO DE SISTEMAS DE IA/descarga.jpeg'  # Update this path
img = image.load_img(img_path, target_size=(256, 256))  # Adjust to 256x256
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)


In [43]:
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
print(f'Predicted class: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
Predicted class: 7


In [44]:
label_list = train_generator.class_indices
# Invertimos el diccionario para que podamos buscar los nombres de las clases por índice
label_list = {v: k for k, v in label_list.items()}
predicted_label = label_list[predicted_class]
print(f'This vehicle is predicted to be a: {predicted_label}')


This vehicle is predicted to be a: helicopter


In [47]:
from flask import Flask, request, render_template
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == "POST":
        # Cargar la imagen del formulario
        file = request.files['file']
        img_path = './uploads/' + file.filename
        file.save(img_path)
        
        # Cargar y preprocesar la imagen
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Hacer predicción
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions[0])
        
        # Convertir índice a clase
        predicted_label = label_list[predicted_class]
        return f'El vehículo es: {predicted_label}'
    return '''
    <!doctype html>
    <title>Subir una imagen</title>
    <h1>Subir una imagen para clasificar el vehículo</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Subir>
    </form>
    '''

if __name__ == "__main__":
    import os
    # Evitar la ejecución en entornos interactivos como Jupyter/IPython
    if os.environ.get("FLASK_ENV") != "development":
        app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1